# T5 SST2 Sentiment Analysis

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `nlp` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `nlp`


## Install Libraries

In [2]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load Model and Tokenizer

In [3]:
# The model we will use is 'michelecafagna26/t5-base-finetuned-sst2-sentiment' from HuggingFace.
# This model is a T5-Base version that has been fine-tuned on the SST2 dataset for sentiment analysis.

from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "michelecafagna26/t5-base-finetuned-sst2-sentiment"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

print(f"Successfully loaded model: {model_name}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Successfully loaded model: michelecafagna26/t5-base-finetuned-sst2-sentiment


## Define Sentiment Prediction Function

In [4]:
# This function uses the specified input prefix "sentiment: " and returns a prediction ('p' or 'n').

def get_sentiment(text):
    inputs = tokenizer("sentiment: " + text, max_length=128, truncation=True, return_tensors="pt").input_ids
    preds = model.generate(inputs)
    decoded_preds = tokenizer.batch_decode(sequences=preds, skip_special_tokens=True)
    return decoded_preds[0]

## Predict Sentiment

In [5]:
text = "This movie is awesome"
prediction = get_sentiment(text)
print(f"Sentence: '{text}'")
print(f"Predicted Sentiment: {prediction}")

Sentence: 'This movie is awesome'
Predicted Sentiment: p


In [6]:
text = "I didn’t like the movie so much"
prediction = get_sentiment(text)
print(f"Sentence: '{text}'")
print(f"Predicted Sentiment: {prediction}")

Sentence: 'I didn’t like the movie so much'
Predicted Sentiment: n


In [7]:
text = "I’m not sure what I think about this movie."
prediction = get_sentiment(text)
print(f"Sentence: '{text}'")
print(f"Predicted Sentiment: {prediction}")

Sentence: 'I’m not sure what I think about this movie.'
Predicted Sentiment: n


In [8]:
text = "Did you like the movie?"
prediction = get_sentiment(text)
print(f"Sentence: '{text}'")
print(f"Predicted Sentiment: {prediction}")

Sentence: 'Did you like the movie?'
Predicted Sentiment: p


## Load SST2 Dataset

In [9]:
from datasets import load_dataset

dataset = load_dataset("SetFit/sst2")
test_dataset = dataset["test"]

print(f"Successfully loaded SST2 test dataset with {len(test_dataset)} examples.")
print(f"Example: {test_dataset[0]}")

README.md:   0%|          | 0.00/378 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/136k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/281k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6920 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Successfully loaded SST2 test dataset with 1821 examples.
Example: {'text': 'no movement , no yuks , not much of anything .', 'label': 0, 'label_text': 'negative'}


## Evaluate Model Accuracy on SST2 Test Set

In [10]:
# This evaluation process involves iterating through the dataset,
# making predictions, and comparing them to the true labels.

from tqdm.auto import tqdm

correct_predictions = 0
total_examples = len(test_dataset)
label_map = {0: "n", 1: "p"}

for example in tqdm(test_dataset, desc="Evaluating"):
    sentence = example["text"]
    true_label_text = label_map[example["label"]]
    predicted_label_text = get_sentiment(sentence)
    if predicted_label_text == true_label_text:
        correct_predictions += 1

accuracy = correct_predictions / total_examples
print(f"\nAccuracy on SST2 Test Set: {accuracy:.4f}")

Evaluating:   0%|          | 0/1821 [00:00<?, ?it/s]


Accuracy on SST2 Test Set: 0.9522


## Check SST2 Dataset Balance

In [11]:
# We will count the number of examples for each label (0 and 1) in the test set
# to determine if the dataset is balanced.

label_counts = {}
for example in test_dataset:
    label = example["label"]
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

print("\nSST2 Test Dataset Label Distribution:")

label_map_print = {0: "negative (n)", 1: "positive (p)"}
for label_id, count in label_counts.items():
    print(f"Label {label_id} ({label_map_print[label_id]}): {count} examples")

count_n = label_counts.get(0, 0)
count_p = label_counts.get(1, 0)
total = count_n + count_p
proportion_n = count_n / total
proportion_p = count_p / total
print(f"\nProportion of Negative (0): {proportion_n:.4f}")
print(f"Proportion of Positive (1): {proportion_p:.4f}")


SST2 Test Dataset Label Distribution:
Label 0 (negative (n)): 912 examples
Label 1 (positive (p)): 909 examples

Proportion of Negative (0): 0.5008
Proportion of Positive (1): 0.4992
